In [1]:
import numpy as np
from sklearn.linear_model import LassoCV, RidgeCV, Lasso, Ridge
from sklearn.gaussian_process.kernels import Matern, RBF
from sklearn.ensemble import RandomForestRegressor

from doc.mse_estimator import ErrorComparer
from doc.data_generation import gen_rbf_X, gen_matern_X, gen_cov_mat
from doc.plotting_utils import gen_model_barplots
from spe.estimators import kfoldcv, kmeanscv, new_y_est, cp_general

from spe.tree import Tree

In [2]:
np.random.seed(1)

In [3]:
## number of realizations to run
niter = 100

## data generation parameters]
gsize=20
n=20**2
p=5
s=5
delta = 0.75
snr = 0.4
tr_frac = .5

noise_kernel = 'matern'
noise_length_scale = 1.
noise_nu = .5

X_kernel = 'matern'
X_length_scale = 5.
X_nu = 2.5

## ErrorComparer parameters
alpha = .05
nboot = 100
k = 5
# lambdas = np.logspace(.01, 10, 10)
max_depth = 2
# models = [Lasso(alpha=.31),RidgeCV(alphas=lambdas), LassoCV(alphas=lambdas)]
models = [
    Tree(max_depth=max_depth), 
    RidgeCV(alphas=[.01,.1, 1.]), 
    RandomForestRegressor(n_estimators=100, max_depth=max_depth)
]

ests = [
    new_y_est,
    new_y_est,
    cp_general,
    kfoldcv, 
    kmeanscv
]
est_kwargs = [
    {'alpha': None,
    'full_refit': False},
    {'alpha': alpha,
    'full_refit': False},
    {'alpha': alpha, 
    'use_trace_corr': False, 
    'nboot': nboot},
    {'k': k},
    {'k': k}
]

## plot parameters
est_names = ["GenCp", "KFCV", "SPCV"]
model_names = ["Decision Tree", "Ridge CV", "Random Forest"]

In [4]:
err_cmp = ErrorComparer()

In [5]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, gsize, nx)
ys = np.linspace(0, gsize, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()
coord = np.stack([c_x, c_y]).T

In [6]:
if noise_kernel == 'rbf':
    Sigma_t = gen_cov_mat(c_x, c_y, RBF(length_scale=noise_length_scale))
elif noise_kernel == 'matern':
    Sigma_t = gen_cov_mat(c_x, c_y, Matern(length_scale=noise_length_scale, nu=noise_nu))
else:
    Sigma_t = np.eye(n)
    
Cov_st = delta*Sigma_t
Sigma_t = delta*Sigma_t + (1-delta)*np.eye(n)

if noise_kernel == 'rbf' or noise_kernel == 'matern':
    Chol_t = np.linalg.cholesky(Sigma_t)
else:
    Chol_t = np.eye(n)

In [7]:
if X_kernel == 'rbf':
    X = gen_rbf_X(c_x, c_y, p)
elif X_kernel == 'matern':
    X = gen_matern_X(c_x, c_y, p, length_scale=X_length_scale, nu=X_nu)
else:
    X = np.random.randn(n,p)

beta = np.zeros(p)
idx = np.random.choice(p,size=s,replace=False)
beta[idx] = np.random.uniform(-1,1,size=s)

In [8]:
tr_idx = np.ones(n, dtype=bool)

# Simulate $Y, Y^* \overset{iid}{\sim} \mathcal{N}(\mu, \Sigma_Y)$

In [9]:
model_errs = []

for model in models:
    errs = err_cmp.compare(
        model,
        ests,
        est_kwargs,
        niter=niter,
        n=n,
        p=p,
        s=s,
        snr=snr, 
        X=X,
        beta=beta,
        coord=coord,
        Chol_y=Chol_t,
        Chol_ystar=None,
        Cov_y_ystar=None,
        tr_idx=tr_idx,
        fair=False,
        piecewise_const_mu=True,
        est_sigma=False,
    )
    model_errs.append(errs)

  0%|          | 0/100 [00:00<?, ?it/s]

/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
  1%|          | 1/100 [00:00<00:47,  2.11it/s]/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` ins

In [10]:
fig = gen_model_barplots(
    model_errs, 
    model_names, 
    est_names, 
    title="General Models: No Shared Noise", 
    has_elev_err=True
)
fig.show()

# Simulate $\begin{pmatrix} Y \\ Y^* \end{pmatrix} \sim \mathcal{N}\left(\begin{pmatrix} \mu \\ \mu \end{pmatrix}, \begin{pmatrix}\Sigma_Y & \Sigma_{Y, Y^*} \\ \Sigma_{Y^*, Y} & \Sigma_{Y}  \end{pmatrix}\right)$

In [11]:
model_errs = []

for model in models:
    errs = err_cmp.compare(
        model,
        ests,
        est_kwargs,
        niter=niter,
        n=n,
        p=p,
        s=s,
        snr=snr, 
        X=X,
        beta=beta,
        coord=coord,
        Chol_y=Chol_t,
        Chol_ystar=None,
        Cov_y_ystar=Cov_st,
        tr_idx=tr_idx,
        fair=False,
        piecewise_const_mu=True,
    )
    model_errs.append(errs)

  0%|          | 0/100 [00:00<?, ?it/s]

/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning:

`fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.

/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning:

`fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.

  1%|          | 1/100 [00:00<01:04,  1.54it/s]/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning:

`fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.

/Users/kevinfry/opt/anaconda3/envs/spe/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning:

`fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.

  2%|▏         | 2/100 [00:01<00:55, 

In [12]:
fig = gen_model_barplots(
    model_errs, 
    model_names, 
    est_names, 
    title="General Models: Shared Structured Noise", 
    has_elev_err=True,
)
fig.show()